### HW2 - Multinomial Logistic Regression & SVMs 

### Training and Evaluations

Connect gdrive for train/test datasets:

In [1]:
from google.colab import drive # import lib
drive.mount("/content/gdrive") # mount gdrive

Mounted at /content/gdrive


Import libraries:

In [2]:
import numpy as np # for np array ops
import matplotlib.pyplot as plt # for visualization purposes
from pandas import * # for reading and parsing .csv files
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Read Data:

In [3]:
sTrainAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/train.csv" # address of train.csv on gdrive 
sTestAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/test.csv" # address of train.csv on gdrive 
data_training = read_csv(sTrainAdd)
data_test = read_csv(sTestAdd)

Preprocess Data:

In [4]:
# Training Data Preprocessing
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_training = DataFrame()
y_training = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_training.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_training = y_training.append(temp1)
  else:
    x_training = x_training.append(temp1)
x_training = x_training.T
y_training = y_training.T
# Test Data Preprocessing
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_test = DataFrame()
y_test = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_test.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_test = y_test.append(temp1)
  else:
    x_test = x_test.append(temp1)
x_test = x_test.T
y_test = y_test.T

<ipython-input-4-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-4-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-4-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-4-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-4-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Encode Datasets:

In [5]:
# One-hot Encoding for Training Dataset:
le = LabelEncoder() # create label object
x_training.diet = le.fit_transform(x_training.diet)
x_training.lived_in = le.fit_transform(x_training.lived_in)
x_training.period = le.fit_transform(x_training.period)
x_training.named_by = le.fit_transform(x_training.named_by)
x_training.taxonomy = le.fit_transform(x_training.taxonomy)
x_training.species = le.fit_transform(x_training.species)
# Limit dataset #
x_training_processed = x_training.drop(columns=['id', 'name', 'length', 'link']) # x_, argument used in training
print(x_training_processed)
# One-hot Encoding for Test Dataset:
x_test.diet = le.fit_transform(x_test.diet)
x_test.lived_in = le.fit_transform(x_test.lived_in)
x_test.period = le.fit_transform(x_test.period)
x_test.named_by = le.fit_transform(x_test.named_by)
x_test.taxonomy = le.fit_transform(x_test.taxonomy)
x_test.species = le.fit_transform(x_test.species)
# Limit dataset #
x_test_processed = x_test.drop(columns=['id', 'name', 'length', 'link']) # x_, argument used in training
print(x_test_processed)

     diet  period  lived_in  taxonomy  named_by  species
0       0     108        25         0        76       61
1       2     108         1        34        11       95
2       1      65         5        20       212      188
3       1      43        24        49        64      181
4       1      21        25        17        22       24
..    ...     ...       ...       ...       ...      ...
241     1      11        13        25       186      184
242     1      10         2        13       161      120
243     0      14         5        79       203      222
244     0      42         1        58        14      182
245     3     112        19        34        70      170

[246 rows x 6 columns]
    diet  period  lived_in  taxonomy  named_by  species
0      1      18        13        24        36       19
1      1      41        17        21         8       51
2      0      24        17        43        26       32
3      0       2        17        30        50        6
4      2    

In [9]:
# Logistic Regression #
# Training and mean_accuracy #
skf = StratifiedKFold(n_splits=5)  #the number of folds is 5
model = LogisticRegression(solver='lbfgs', max_iter=1000)
accuracy_scores = []
for train_index, test_index in skf.split(x_training_processed, y_training):
  X_train, X_test = x_training_processed.iloc[train_index], x_training_processed.iloc[test_index]
  y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index]  
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
np.savetxt('training_LR.csv', model.predict(x_training_processed), delimiter=',', fmt='%s') # save result to text
np.savetxt('test_LR.csv', model.predict(x_test_processed), delimiter=',', fmt='%s') # save result to text
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Logistic Regression - Mean accuracy:", mean_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel

Logistic Regression - Mean accuracy: 0.6464489795918367


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Logistic Regression #
# Calculate Logistic Regression f1 score #
f1_scores_LR = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_, y):
    X_train, X_test = x_.iloc[train_index], x_.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index] 
    model.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = model.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score = f1_score(y_test, y_pred, average='weighted')
    f1_scores_LR.append(fold_f1_score)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score = np.mean(f1_scores_LR)
print("Logistic Regression - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score)

In [ ]:
# SVM #
# Training and mean_accuracy #
skf = StratifiedKFold(n_splits=5)  #the number of folds is 5
accuracy_scores = []
for train_index, test_index in skf.split(x_, y):
  X_train, X_test = x_.iloc[train_index], x_.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]   
  svm = SVC(kernel='linear', C=1.0, random_state=42)
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("SVM - Mean accuracy:", mean_accuracy)

ValueError: ignored

In [ ]:
# SVM #
# Calculate SVM f1 score #
f1_scores_SVM = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_, y):
    X_train, X_test = x_.iloc[train_index], x_.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index] 
    svm.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = svm.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score = f1_score(y_test, y_pred, average='weighted')
    f1_scores_SVM.append(fold_f1_score)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score = np.mean(f1_scores_SVM)
print("SVM - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score)